# **WEB SCRAPING API**

**Kelas 2024A**

Rebriane Atitha G. (039)

Nabilah Hilmi R. (053)

Kekila Akmal N. (144)

Web Scraping dengan API:
1. Identifikasi Situs Web atau Layanan yang Menyediakan API
2. Dapatkan Kunci API (API Key)
3. Buat Permintaan ke API
4. Urai (Parse) Data Respons
5. Simpan Data

**Inisialisasi dan Impor Pustaka**

In [ ]:
import requests
import json
import pandas as pd
import time
api_key = '41984B7715C8A57676000E24258DC3E3'

**Mengambil Daftar Aplikasi**

In [ ]:
get_app_list_url = "http://api.steampowered.com/ISteamApps/GetAppList/v2/"
response = requests.get(get_app_list_url)
data = response.json()
app_list = data["applist"]["apps"]
print(f"Total apps: {len(app_list)}")

Total apps: 266722


**Menyimpan Daftar Aplikasi ke File JSON**

In [ ]:
with open("response.json", "w", encoding="utf-8") as f:
    json.dump(app_list, f, indent=4, ensure_ascii=False)


**Mengambil Detail Setiap Game**

In [ ]:
with open("response.json", "r", encoding="utf-8") as f:
    data = json.load(f)

games_data = []
app_details_url = "http://store.steampowered.com/api/appdetails"

for app in data[:100]:
    appid = app["appid"]
    name = app["name"]

    detail_url = f"{app_details_url}?appids={appid}&l=english"
    try:
        res = requests.get(detail_url, timeout=10)
        if res.headers.get("Content-Type","").startswith("application/json"):
            app_info = res.json().get(str(appid), {})
        else:
            app_info = {}

        if app_info.get("success") and "data" in app_info:
            game = app_info["data"]
            genres = ", ".join([g["description"] for g in game.get("genres", [])])
            price = game.get("price_overview", {}).get("final_formatted", "Gratis")
        else:
            genres, price = "", "Unknown"

        games_data.append([appid, name, genres, price])

    except Exception as e:
        print(f"Error {appid}: {e}")
        games_data.append([appid, name, "", "Error"])

    time.sleep(0.3)


**Menyimpan Data ke File CSV**

In [ ]:
import pandas as pd
df = pd.DataFrame(
    games_data,
    columns=['AppID', 'Name', 'Genres', 'Price']
)
display(df)
df.to_csv("steam_games_sample.csv", index=False, encoding="utf-8")
print("\nData berhasil disimpan ke file steam_games_sample.csv")